In [163]:
# pip install pyramid

In [164]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyramid
from sklearn import metrics

In [237]:
dataset1=pd.read_csv('MONTH-WISE_FGAS-660A-16-00087.csv')
dataset1['SIHD_INV_DATE']=pd.to_datetime(dataset1['SIHD_INV_DATE'])
dataset1.head()

,SIHD_INV_DATE,QTY
0,2019-04-30,9030
1,2019-05-31,9786
2,2019-06-30,7434
3,2019-07-31,8232
4,2019-08-31,4662


In [238]:
from statsmodels.tsa.stattools import adfuller
dftest = adfuller(dataset1['QTY'], autolag = 'AIC')
print("1. ADF : ",dftest[0])
print("2. P-Value : ", dftest[1])
print("3. Num Of Lags : ", dftest[2])
print("4. Num Of Observations Used For ADF Regression:",      dftest[3])
print("5. Critical Values :")
for key, val in dftest[4].items():
    print("\t",key, ": ", val)
if dftest[1]< 0.05:
    print("Data is stationary")
else:
    print("Data is non-stationary")

1. ADF :  -2.6782322467522555
2. P-Value :  0.07788137783626346
3. Num Of Lags :  2
4. Num Of Observations Used For ADF Regression: 25
5. Critical Values :
	 1% :  -3.7238633119999998
	 5% :  -2.98648896
	 10% :  -2.6328004
Data is non-stationary


In [240]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(dataset1['SIHD_INV_DATE'],dataset1['QTY'],test_size=0.2,shuffle=False,random_state=123)
y_train,y_test

(0     9030
 1     9786
 2     7434
 3     8232
 4     4662
 5     5712
 6     4116
 7     5334
 8     4074
 9     3780
 10    6384
 11    6426
 12       0
 13     630
 14    3696
 15    4914
 16    3066
 17    5880
 18    6384
 19    5964
 20    3864
 21    4242
 Name: QTY, dtype: int64,
 22     8610
 23     8274
 24     6006
 25      966
 26     4092
 27    10920
 Name: QTY, dtype: int64)

In [241]:
def timeseries_evaluation_metrics_func(y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MSE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true, y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}', end='\n\n')

In [242]:
from pmdarima import auto_arima
stepwise_fit = auto_arima(y_train,start_p=1, start_q=1,
                          d=None, trace=True,error_action='ignore',suppress_warnings=True, stepwise=True)
print(stepwise_fit.summary())

Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=404.689, Time=0.04 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=407.282, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=402.141, Time=0.06 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=403.048, Time=0.11 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=444.635, Time=0.02 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=404.059, Time=0.09 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=405.968, Time=0.16 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=406.697, Time=0.03 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0] intercept
Total fit time: 0.534 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                   22
Model:               SARIMAX(1, 0, 0)   Log Likelihood                -198.070
Date:                Sat, 08 Jan 2022   AIC                            402.141
Time:                        12:54:59   BIC 

In [243]:
from statsmodels.tsa.arima.model import ARIMA

In [244]:
model = ARIMA(y_train, order=(1,0,0))

In [245]:
fitted_model = model.fit()

In [246]:
fitted = fitted_model.forecast(13, alpha=0.5)
fitted, y_test

(22    4666.245076
 23    4898.770035
 24    5026.214900
 25    5096.066298
 26    5134.351230
 27    5155.334861
 28    5166.835802
 29    5173.139366
 30    5176.594294
 31    5178.487909
 32    5179.525782
 33    5180.094632
 34    5180.406413
 Name: predicted_mean, dtype: float64,
 22     8610
 23     8274
 24     6006
 25      966
 26     4092
 27    10920
 Name: QTY, dtype: int64)

In [247]:
np.mean(fitted)

5093.235892107418

In [248]:
np.mean(y_test)

6478.0